In [4]:
from bs4 import BeautifulSoup as bs
import requests
from tqdm import tqdm 
import json
import os
import datetime
import re
import glob
import csv
import time
import pandas as pd
from nltk.stem import *
from collections import Counter
from functools import reduce
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk import tokenize
import datatable as dt   
import csv


#To read and store dictionary
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\auror\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [99]:
#TODO
#EDA
#do Lemmatization in the Data Cleaning 
#Create all the missing tsv
#Clean and comment the code
#CHECK EFFICIENCY 1.2 and 1.3
#Parse list before saving into
# finire di aggiungere labels al punto 4 (placeAdress,city)

### 1. Data collection
## 1.1. Get the list of places

In [ ]:
total_link = []
s = requests.Session()
for n in tqdm(range(1,401)):
    url = f'https://www.atlasobscura.com/places?page={n}&sort=likes_count'
    result = s.get(url)
    soup = bs(result.text)
    puf = soup.find_all("a", {'class': 'content-card content-card-place'})
    for x in puf:
        total_link.append(x['href'])


In [ ]:
f = open('total_link.txt',"r")
total_link = f.read().split("\n")

## 1.2. Crawl places

In [ ]:
def downloadPage(start,end, array):
    count_link = ((start-1)*18)+1
    count_page = start
    
    parent_dir = f'./all_Pages'
    s = requests.Session()
    header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    for x in tqdm(range(start, end)):
        if count_link%10 == 0:
            s = requests.Session()
        if count_link%18 == 1:
            path = os.path.join(parent_dir, f"folder_{count_page}")
            os.mkdir(path)
        for y in range(18):
            url = f'https://www.atlasobscura.com{array[count_link-1]}'
            name_file = f'location_{count_link}'
            name_folder = f'folder_{count_page}'
            with open(f'./all_Pages/{name_folder}/{name_file}.html', 'w', encoding='utf8') as fp:
                req = s.get(url, headers = header)
                fp.write(req.text)
                if req.status_code != 200:
                     time.sleep(120)
                     req = s.get(url, headers = header)
                fp.write(s.get(url).text)
            if count_link%18 == 0:
                count_page += 1
            count_link += 1
           
        

## 1.3 Parse downloaded pages

In [ ]:

def save_TSV():
    os.mkdir("TSV Files")
    for x in os.listdir("all_Pages"):
        for y in os.listdir(f'all_Pages/{x}'):
            with open(f'./all_Pages/{x}/{y}', encoding='utf8') as f:
                p = f.read()
                soup =  bs(p)
                pageAttribute = []
                placeName = findPlaceName(soup)
                placeTags = findPlaceTags(soup)
                numPeopleVisited = findNumPeopleVisited(soup)
                numPeopleWant = findNumPeopleWant(soup)
                placeDesc = findDescription(soup)
                placeShortDesc = findShortDescription(soup)
                placeNearby = findNearbyPlaces(soup)
                placeAddress = findAddress(soup)
                placeAlt, placeLong = findCordinates(soup)
                placeEditors = findPostEditors(soup)
                placePubDate = findPublishingDate(soup)
                placeRelatedList = findPlaceNear(soup)
                placeRelatedPlaces = findRelatedPlaces(soup)
                placeURL = findPageURL(soup)
                with open(f'./TSV Files/{y[:-5]}.tvs', 'wt', encoding='utf8') as fp:
                    csv.writer(fp, delimiter='\t').writerow([placeName, placeTags, numPeopleVisited, numPeopleWant, placeDesc, placeShortDesc, placeNearby, placeAddress, placeAlt, placeLong, placeEditors, placePubDate, placeRelatedList, placeRelatedPlaces, placeURL])
            

In [ ]:

def download_TSV():
    os.mkdir("TSV Files")
    a = 0
    for path in tqdm(glob.glob(r"all_Pages/*/*")):
        with open(path, encoding='utf8') as f:
                    a += 1
                    p = f.read()
                    soup =  bs(p)
                    pageAttribute = []
                    placeName = findPlaceName(soup)
                    placeTags = findPlaceTags(soup)
                    placeTags = ",".join(placeTags)
                    numPeopleVisited = findNumPeopleVisited(soup)
                    numPeopleWant = findNumPeopleWant(soup)
                    placeDesc = findDescription(soup)
                    placeDesc = " ".join(placeDesc)
                    placeShortDesc = findShortDescription(soup)
                    placeNearby = findNearbyPlaces(soup)
                    placeNearby = ",".join(placeNearby)
                    placeAddress = findAddress(soup)
                    placeAlt, placeLong = findCordinates(soup)
                    placeEditors = findPostEditors(soup)
                    placeEditors = ",".join(placeEditors)
                    placePubDate = findPublishingDate(soup)
                    placeRelatedList = findPlaceNear(soup)
                    placeRelatedList = ",".join(placeRelatedList)
                    placeRelatedPlaces = findRelatedPlaces(soup)
                    placeRelatedPlaces = ",".join(placeRelatedPlaces)
                    placeURL = findPageURL(soup)
                    with open(f'./TSV Files/{a}.tvs', 'wt', encoding='utf8') as fp:
                        csv.writer(fp, delimiter='\t').writerow([placeName, placeTags, numPeopleVisited, numPeopleWant, placeDesc, placeShortDesc, placeNearby, placeAddress, placeAlt, placeLong, placeEditors, placePubDate, placeRelatedList, placeRelatedPlaces, placeURL])
        
        

In [ ]:
download_TSV()

In [ ]:
def findPlaceName(soup):
    placeName = soup.find("h1", {"class": "DDPage__header-title"})
    if placeName != None:
        placeName = placeName.text
    #placeName = re.sub('[A-Za-z0-9_.,! "]*' ,'',placeName)
    return placeName

def findPlaceTags(soup):
    tags = []
    placeTags = soup.find_all("a", {"class": "itemTags__link js-item-tags-link"})
    for tag in placeTags:
        t = tag.text.replace("\n", "")
        #t = re.sub('[A-Za-z0-9 _.,!"]*','',t)
        tags.append(t)
    return tags

def findNumPeopleVisited(soup):
    peopleVisited = soup.find_all("div", {"class": "title-md item-action-count"})
    if len(peopleVisited) > 0:
        peopleVisited = int(peopleVisited[0].text)
    return peopleVisited

def findNumPeopleWant(soup):
    peopleVisited = soup.find_all("div", {"class": "title-md item-action-count"})
    if len(peopleVisited) > 0:
        peopleVisited = int(peopleVisited[1].text)
    return peopleVisited

def findDescription(soup):
    all_description = []
    descriptions = soup.find_all("div", {"class": "DDP__body-copy"})
    for description in descriptions:
        d = description.text.replace("\n","")
        all_description.append(d)
    return all_description

def findShortDescription(soup):
    shortDescription = soup.find("h3", {"class": "DDPage__header-dek"})
    if shortDescription != None:
        shortDescription = shortDescription.text.replace("\n", "")
    return shortDescription

def findNearbyPlaces(soup):
    nearPlaces = []
    nearbyPlaces = soup.find_all("div", {"class": "DDPageSiderailRecirc__item-title"})
    if nearbyPlaces != None:
        for place in nearbyPlaces:
            p = place.text.replace("\n","")
            nearPlaces.append(p)
        #Convert the list to set, and then back again to list to remove repetition
    return set(nearPlaces)

def findAddress(soup):
    strings = []
    adress_strings = soup.find("address", {"class": "DDPageSiderail__address"})
    if adress_strings != None:
        adress_strings = adress_strings.find("div")
        for info in adress_strings:
            s = info.text.replace("\n", "")
            if s != "":
                strings.append(s)
        if len(strings) > 3:
            return " ".join(strings[:3])
    else:
        return " "

def findCordinates(soup):

    cordinates = soup.find("div", {"class":"DDPageSiderail__coordinates js-copy-coordinates"})
    if cordinates != None:
        return cordinates.text.replace("\n", "").replace(" ","").split(",")
    return " ", " "

def findPostEditors(soup):
    all_editors =[]
    editors = soup.find_all("a", {"class":"DDPContributorsList__contributor"})         
    for person in editors:
        s = person.text.replace("\n", "")
        all_editors.append(s)
    return all_editors

def findPublishingDate(soup):
    #Pick the right info
    dateString = soup.find("div", {"class":"DDPContributor__name"}) 
    #Let's clean the string
    if dateString != None:
        s = dateString.text.replace("\n", "")
        #Let's modify it for the right format of datetime
        split = s.split()
        #Let's convert the string Month into the corrispondent number by using "strptime()" 
        split[0] = str(datetime.datetime.strptime(split[0], '%B').month)
        #My format
        format = "%m %d, %Y"
        #Convert from String to datetime
        date = datetime.datetime.strptime(" ".join(split), format)
        return date
    else:
        return ""

def findPlaceNear(soup):
    lists =[]
    relatedLists = soup.find("div", {"data-gtm-template":"DDP Footer Recirc Nearby"})
    if relatedLists != None:
        relatedLists = relatedLists.find_all("h3", {"class":"Card__heading --content-card-v2-title js-title-content"})
        for list in relatedLists:
            s = list.text.replace("\n", "")
            #s = re.sub('[A-Za-z0-9 _.,!"]*','',s)
            lists.append(s)
    return lists

def findRelatedPlaces(soup):
    lists =[]
    relatedLists = soup.find("div", {"data-gtm-template":"DDP Footer Recirc Related"})
    if relatedLists != None:
        relatedLists = relatedLists.find_all("h3", {"class":"Card__heading --content-card-v2-title js-title-content"})
        for list in relatedLists:
            s = list.text.replace("\n", "")
            #s = re.sub('[A-Za-z0-9 _.,!"]*','',s)
            lists.append(s)
    return lists

def findPageURL(soup):
    numVisitedPeople = soup.find("link", {"rel":"canonical"})
    return numVisitedPeople['href']



## Load all the tsv file into one pandas dataframe

We decided to save all the .tsv file as one joined .csv where every file rappresents one row of the .csv file, so that we can work and access the data without having to read 7200 tsv file each time. The data are stored in the "tsv_dataframe"

This mean that the function load_tsv() has only been executed once. The other iteration we just read the .csv file

In [ ]:

def load_tsv():
    tsv = []
    dtypes = {}
    for x in tqdm(os.listdir("TSV Files")):
        df = pd.read_csv(f'TSV Files/{x}',
            sep="\t",
            header=None,
            names=["placeName", "placeTags", "numPeopleVisited", "numPeopleWant", "placeDesc", "placeShortDesc", "placeNearby","placeAdress", "placeAlt", "placeLong", "placeEditors","placePubDate", "placeRelatedList", "placeRelatedPlace", "placeURL"])
        tsv.append(df)

    return pd.concat(tsv)

In [5]:
# data = load_tsv()
data = pd.read_csv("tsv_dataframe.csv",index_col=0)

#Reset Index
data.reset_index(inplace = True, drop=True)

In [ ]:
data.info()

In [23]:
data.head()

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAdress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedList,placeRelatedPlace,placeURL
0,City Hall Station,"subways,subterranean,infrastructure,subterrane...",1834,8603,The first New York City subway was built and o...,A beautiful and abandoned New York subway stat...,"The Sugar House Prison Window,Tunnel Number 3,...","31 Centre St New York, New York, 10007 United ...",40.7134,-74.0046,"Rebekah Otto,Annetta Black,Allan,charding407,w...",2010-05-08 00:00:00,"African Burial Ground National Monument,Tunnel...","Crystal Palace Subway,Moscow Metro Stations,Ro...",https://www.atlasobscura.com/places/city-hall-...
1,The Winchester Mystery House,"follies and grottoes,outsider architecture,ecc...",3723,5209,In 1886 an eccentric woman named Sarah Winches...,A peculiar mansion built by the troubled heir ...,"Rosicrucian Park,Rosicrucian Egyptian Museum,S...","525 South Winchester Boulevard San Jose, Calif...",37.3189,-121.9506,"mbison,Avoiding Regret,PrestonW,Collin,Destina...",2008-11-21 00:00:00,"Santana Row Chess Plaza,Rosicrucian Park,Rosic...","Casa de Piedra (Stone House),Portmeirion Villa...",https://www.atlasobscura.com/places/winchester...
2,Center for Puppetry Arts,"sesame street,puppets,museums",705,1027,"If you know your Gonzo from your Gelflings, yo...",The largest puppetry museum in the USA houses ...,"Breman Museum,The Consulate,Atlanta Monetary M...","1404 Spring St Atlanta, Georgia United States",33.7927,-84.3896,"kendallmbailey,Martin,andydoesit,We All Pod Do...",2018-11-12 00:00:00,"Breman Museum,Atlanta Monetary Museum,The Cons...","Portland Puppet Museum,Museu da Marioneta (Mus...",https://www.atlasobscura.com/places/center-for...
3,Amityville Horror House,"hoaxes and pseudoscience,haunted,film location...",367,831,"On a cold November night in 1974, Amityville, ...",The site of a tragic massacre that spawned a m...,"The Long Island Puppet Theater,Pilgrim Psychia...","112 Ocean Avenue Amityville, New York, 11701 U...",40.6665,-73.4143,"EricGrundhauser,MandrewPatinkin,lendog666,Mart...",2013-10-17 00:00:00,"The Long Island Puppet Theater,Fire Island Lig...","Conanicut Island Lighthouse,Lovell Health Hous...",https://www.atlasobscura.com/places/amityville...
4,Yuma Territorial Prison,"prisons,crime and punishment",422,833,"On July 1, 1876, the first seven inmates enter...",Walk through the actual strap iron cells and s...,"Valley of the Names,Felicity, California: Cent...","1 Prison Hill Rd Yuma, Arizona, 85364 United S...",32.7266,-114.6144,"desertfairy,mbison,CorpseLady84,Avoiding Regre...",2010-04-01 00:00:00,"Felicity, California: Center of the World,Vall...","Buckley's Cave,Palazzo d'Avalos,Oxford Castle ...",https://www.atlasobscura.com/places/yuma-terri...


In [ ]:
data.sort_values('numPeopleWant')

## Data Cleaning

- remove punctuation 
- remove stopwords 
- stemming 

In [5]:


def remove_punctuations(string):
    # first we remove the punctuations
    # in order to do it we need to tekenize the string with the function tokenize and then applying the function RegexpTokenizer
    return RegexpTokenizer(r'\w+').tokenize(string)


def stemming(string):
    # now we move forward with the stemming
    porter = PorterStemmer()
    string_stem=[porter.stem(word) for word in string]
    # we can now return the cleaned string 
    return string_stem

def remove_stopwords(string):
    # after this we can now remove all the stopwords in each word in string_t
    return  [word for word in string if not word.lower() in set(stopwords.words("english"))]
    # now we move forward with the stemming

def cleaning(string):
    #I apply all the function for cleaning the string
    string = string.lower()
    string = remove_punctuations(string)
    string = remove_stopwords(string)
    string = stemming(string)

    #return a list containing all the words of the original string after the cleaning
    return string

## 2.1 

In [6]:
#The first dictionary where every key is a word and it's value is the corrispondant number to that word

def createFirstDic(data):
    dic = {}
    value = 1
    for description in tqdm(data["placeDesc"]):
        #Cleaning the data
        description = cleaning(description)
        for word in description:
            if word in dic.keys():
                continue
            else:
                dic[word] = value
                value += 1
    return dic

In [7]:
dic1 = createFirstDic(data)

100%|██████████████████████████████████████████████████████████████████████████████| 7200/7200 [09:32<00:00, 12.58it/s]


In [ ]:
len(dic1)

## 2.1.1 Creating your index

In [ ]:
def createSecondDic(data, dic1):
    dic2 = {}
    a = 0
    for description in data["placeDesc"]:
        #Cleaning the data
        description= cleaning(description)
        for word in description:
            if dic1[word] in dic2.keys():
                dic2[dic1[word]].add(data.placeName[a])
            else:
                dic2[dic1[word]] = set([data.placeName[a]])
        a += 1
    return dic2

    

In [ ]:
dic2 = createSecondDic(data,dic1)

In [17]:
def save_dic(dic, name):
    with open(f'{name}.pkl', 'wb') as f:
        pickle.dump(dic, f)

In [6]:
def read_dic(name):
    with open(f'{name}.pkl', 'rb') as f:
        loaded_dict = pickle.load(f)
        return loaded_dict

In [19]:
dic1 = read_dic("dic1")

## 2.1 Execute the query

In [7]:
def query(list):
    s = dic2[dic1[list[0]]]
    for x in range(1, len(list)):
        s.intersection(dic2[dic1[list[x]]])
    return s

In [ ]:
s = query(["class", "including"])

res_query = data[data['placeName'].isin(list(s))]

In [ ]:
res_query.head()

## 2.2 Conjunctive query & Ranking score

In [ ]:
def stem_text(txt):
    return [word for word in txt.split(' ')]

In [ ]:
data['list_words'] = data.placeDesc.apply(lambda row: stem_text(row))

In [ ]:

vectorizer = TfidfVectorizer(use_idf=True)
x =vectorizer.fit_transform(data.placeDesc).todense()
df = pd.DataFrame(x, columns = 
vectorizer.get_feature_names())
df
#vectorizer.vocabulary_

In [ ]:
def createInvertedIndex(data, dic1):
    dic2 = {}
    a = 0
    for description in data["placeDesc"]:
        description = description.replace('\'',"")
        description = description.replace('[',"")
        description = description.replace(']',"")
        description = description.lower()
        for word in description.split():
            if dic1[word] in dic2.keys():
                dic2[dic1[word]].add(data.placeName[a])
            else:
                b = []
                dic2[dic1[word]] = set([data.placeName[a]])
            # print(dic2)
        a += 1
    return dic2

## 3. Define a new score!

In [86]:
query_user = list(input("Insert the words that you want to be found in the description's places: ").split())

Insert the words that you want to be found in the description's places: church


In [87]:
#first of all we execute the search engine as in 2.1 with the query given by the user
dic1 = read_dic("dic1")
dic2 = read_dic("dic2")
s = query(query_user)
res_query = data[data['placeName'].isin(list(s))]
res_query.reset_index(inplace = True, drop=True) 

In [88]:
n = int(input('What is the top rank visualization that you want to have? \nInsert "1" if you prefer that is by number of tourists. \nInsert "2" if you prefer that is by number of people that want to visit the site.\n'))

What is the top rank visualization that you want to have? 
Insert "1" if you prefer that is by number of tourists. 
Insert "2" if you prefer that is by number of people that want to visit the site.
2


In [89]:
k = int(input('Insert k the number of places that you want to visualize:')) 
m = int(input('Insert "1" if you want to see the k most popular places or insert "2" if you want to see the less famous places:'))

Insert k the number of places that you want to visualize:50
Insert "1" if you want to see the k most popular places or insert "2" if you want to see the less famous places:1


In [90]:
def new_score(res_query,n,k,m):
    scores = []
    if n == 1:
        by = res_query['numPeopleVisited']
    else:
        by = res_query['numPeopleWant']
    max_value = by.max()
    for i in range(res_query.shape[0]):
        scores.append(round(by[i]/max_value,4))
    res_query.insert(res_query.shape[1], "scores", scores)
    if m == 1:
        res_query = res_query.sort_values(by=['scores'],ascending=False)
    else:
        res_query = res_query.sort_values(by=['scores'],ascending=True)
        
    return res_query.head(k)

In [91]:
resquery = new_score(res_query,n,k,m)
resquery

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAdress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedList,placeRelatedPlace,placeURL,scores
93,Dream House,"optical oddities,eccentric homes,music,outside...",765,6007,"When walking down Church Street in Tribeca, ke...","La Monte Young and Marian Zazeela's ""Dream Hou...","Aretha Franklin Subway Tributes,Farm.One,The '...","275 Church St New York, New York, 10013 United...",40.7185,-74.0048,"seanmattison,mbison,Gray,Sofy,erosika,Blindcol...",2009-09-22 00:00:00,"Aretha Franklin Subway Tributes,Farm.One,The '...","Callejon de Hamel,Casa Neverlandia,Casa de Azu...",https://www.atlasobscura.com/places/dream-house,1.0000
541,The Ruins of St. Dunstan-in-the-East,"gardens,world war ii,churches,plants,military,...",1884,5754,The church of St.Dunstan-in-the-East has survi...,One of the few remaining casualties of the Lon...,"Billingsgate Roman House and Baths,Pudding Lan...",St. Dunstan-in-the-East St. Dunstan's Hill Lon...,51.5097,-0.0827,"Luke J Spencer,Goksu,Kas,adamjerodandrews,Mons...",2015-03-12 00:00:00,"Billingsgate Roman House and Baths,Philpot Lan...","Kaiser Wilhelm Memorial Church,Ruins of Holyro...",https://www.atlasobscura.com/places/the-ruins-...,0.9579
92,Roosevelt Island Smallpox Hospital Ruins,"smallpox,quarantine,abandoned hospitals,abando...",1709,5153,Few diseases have had a greater impact on the ...,A crumbling hospital from the 1850s on Rooseve...,"United Nations Delegates Lounge,Roosevelt Isla...","Roosevelt Island New York, New York, 10044 Uni...",40.7515,-73.9596,"Rebekah Otto,Sharon Blackstone,spavalice,mbiso...",2010-01-13 00:00:00,"Roosevelt Island Cat Sanctuary,Strecker Memori...","Ruins of Lazaretto de Isla de Cabras,Glenn Dal...",https://www.atlasobscura.com/places/roosevelt-...,0.8578
262,The Hardy Tree,"ao loves halloween 2022,gravestones,trees,chur...",859,4042,"Inside an ancient London churchyard, an ash tr...",This churchyard arbor is surrounded by hundred...,"Platform 9 3/4,Word on the Water,British Libra...","Pancras Road London, England, NW1 1UL United K...",51.5349,-0.1309,"sarahvjanet,Pengwn,SEANETTA,giloscope,Stevren,...",2015-07-03 00:00:00,"Word on the Water,Platform 9 3/4,British Libra...",Convento de la Santa Cruz (Convent of Santa Cr...,https://www.atlasobscura.com/places/the-hardy-...,0.6729
481,Santa Maria della Concezione Crypts,"capuchins,marquis de sade,mark twain,ossuaries...",2070,3529,"In 1775, the Marquis de Sade wrote of it, “I h...",The crypts of Capuchin friars decorated with t...,"Saint Victoria's Incorruptible Body,Keats-Shel...","Via Vittorio Veneto, 27 Rome, 00187 Italy",41.9047,12.4882,"Dylan,mshkrvn,Annetta Black,The Dubious Hausfr...",2009-08-09 00:00:00,"Keats-Shelley Memorial House,Zuccari Palace,Sa...","The Czech's Capuchin Crypt,Santa Maria dell'Or...",https://www.atlasobscura.com/places/santa-mari...,0.5875
501,International Church of Cannabis,"cannabis,marijuana,drugs,religion,churches,sac...",607,3206,A Lutheran church has stood at 400 S Logan Str...,A technicolor place of worship for Elevationis...,"Fifty-Two 80's,Cheesman Park,History Colorado ...","400 S Logan St Denver, Colorado United States",39.7091,-104.9823,"lewblank,Simon Jones,HighPriest,AngieSmangie,c...",2017-07-20 00:00:00,"Cheesman Park,Fifty-Two 80's,History Colorado ...","Blackfriars Friary,Santuario de Atotonilco,Her...",https://www.atlasobscura.com/places/internatio...,0.5337
523,House of Nicolas Flamel,"alchemy,occult,magic,restaurants,houses,homes",646,3042,Strange symbols adorn the architecture at 51 r...,The oldest stone house in Paris was built by i...,"Dans le Noir?,The Relic Crypt of St. Helena at...","51 rue de Montmorency Paris, 75003 France",48.8635,2.3531,"Allison,kez7585,Peccable,Traveling Maven,Scott...",2014-01-26 00:00:00,The Relic Crypt of St. Helena at Église Saint-...,"Boleskine House,Porta Alchemica,Faustův Dům (F...",https://www.atlasobscura.com/places/house-of-n...,0.506

In [92]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")

In [93]:
resquery.reset_index(inplace = True, drop=True)
city=[]
country=[]
for i in range(resquery.shape[0]):
    lat = resquery.placeAlt[i]
    lon = resquery.placeLong[i]
    location = geolocator.reverse(str(lat)+","+str(lon))
    address = location.raw['address']
    city.append(address.get('state', ''))
    country.append(address.get('country', ''))
    
    
#print(city,country)
resquery.insert(resquery.shape[1], "city/state", city)
resquery.insert(resquery.shape[1], "country", country)
resquery


,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAdress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedList,placeRelatedPlace,placeURL,scores,city/state,country
0,Dream House,"optical oddities,eccentric homes,music,outside...",765,6007,"When walking down Church Street in Tribeca, ke...","La Monte Young and Marian Zazeela's ""Dream Hou...","Aretha Franklin Subway Tributes,Farm.One,The '...","275 Church St New York, New York, 10013 United...",40.7185,-74.0048,"seanmattison,mbison,Gray,Sofy,erosika,Blindcol...",2009-09-22 00:00:00,"Aretha Franklin Subway Tributes,Farm.One,The '...","Callejon de Hamel,Casa Neverlandia,Casa de Azu...",https://www.atlasobscura.com/places/dream-house,1.0000,New York,United States
1,The Ruins of St. Dunstan-in-the-East,"gardens,world war ii,churches,plants,military,...",1884,5754,The church of St.Dunstan-in-the-East has survi...,One of the few remaining casualties of the Lon...,"Billingsgate Roman House and Baths,Pudding Lan...",St. Dunstan-in-the-East St. Dunstan's Hill Lon...,51.5097,-0.0827,"Luke J Spencer,Goksu,Kas,adamjerodandrews,Mons...",2015-03-12 00:00:00,"Billingsgate Roman House and Baths,Philpot Lan...","Kaiser Wilhelm Memorial Church,Ruins of Holyro...",https://www.atlasobscura.com/places/the-ruins-...,0.9579,England,United Kingdom
2,Roosevelt Island Smallpox Hospital Ruins,"smallpox,quarantine,abandoned hospitals,abando...",1709,5153,Few diseases have had a greater impact on the ...,A crumbling hospital from the 1850s on Rooseve...,"United Nations Delegates Lounge,Roosevelt Isla...","Roosevelt Island New York, New York, 10044 Uni...",40.7515,-73.9596,"Rebekah Otto,Sharon Blackstone,spavalice,mbiso...",2010-01-13 00:00:00,"Roosevelt Island Cat Sanctuary,Strecker Memori...","Ruins of Lazaretto de Isla de Cabras,Glenn Dal...",https://www.atlasobscura.com/places/roosevelt-...,0.8578,New York,United States
3,The Hardy Tree,"ao loves halloween 2022,gravestones,trees,chur...",859,4042,"Inside an ancient London churchyard, an ash tr...",This churchyard arbor is surrounded by hundred...,"Platform 9 3/4,Word on the Water,British Libra...","Pancras Road London, England, NW1 1UL United K...",51.5349,-0.1309,"sarahvjanet,Pengwn,SEANETTA,giloscope,Stevren,...",2015-07-03 00:00:00,"Word on the Water,Platform 9 3/4,British Libra...",Convento de la Santa Cruz (Convent of Santa Cr...,https://www.atlasobscura.com/places/the-hardy-...,0.6729,England,United Kingdom
4,Santa Maria della Concezione Crypts,"capuchins,marquis de sade,mark twain,ossuaries...",2070,3529,"In 1775, the Marquis de Sade wrote of it, “I h...",The crypts of Capuchin friars decorated with t...,"Saint Victoria's Incorruptible Body,Keats-Shel...","Via Vittorio Veneto, 27 Rome, 00187 Italy",41.9047,12.4882,"Dylan,mshkrvn,Annetta Black,The Dubious Hausfr...",2009-08-09 00:00:00,"Keats-Shelley Memorial House,Zuccari Palace,Sa...","The Czech's Capuchin Crypt,Santa Maria dell'Or...",https://www.atlasobscura.com/places/santa-mari...,0.5875,Lazio,Italia
5,International Church of Cannabis,"cannabis,marijuana,drugs,religion,churches,sac...",607,3206,A Lutheran church has stood at 400 S Logan Str...,A technicolor place of worship for Elevationis...,"Fifty-Two 80's,Cheesman Park,History Colorado ...","400 S Logan St Denver, Colorado United States",39.7091,-104.9823,"lewblank,Simon Jones,HighPriest,AngieSmangie,c...",2017-07-20 00:00:00,"Cheesman Park,Fifty-Two 80's,History Colorado ...","Blackfriars Friary,Santuario de Atotonilco,Her...",https://www.atlasobscura.com/places/internatio...,0.5337,Colorado,United States
6,House of Nicolas Flamel,"alchemy,occult,magic,restaurants,houses,homes",646,3042,Strange symbols adorn the architecture at 51 r...,The oldest stone house in Paris was built by i...,"Dans le Noir?,The Relic Crypt of St. Helena at...","51 rue de Montmorency Paris, 75003 France",48.8635,2.3531,"Allison,kez7585,Peccable,Traveling Maven,Scott...",2014-01-26 00:00:00,The Relic Crypt of St.

In [94]:
print(resquery.shape)

(50, 18)


In [98]:
import plotly.express as px
import plotly.graph_objs as go

fig = px.scatter_geo(resquery,lat="placeAlt", lon="placeLong",labels=resquery['placeAdress'], size="numPeopleVisited", color="country",
                     projection="natural earth",hover_name="placeName")
fig.show()